Task 1: Pull Descriptive Statistics with a CTE

Using a CTE and the "subscriptions" and "products" tables, calculate the:
- minimum monthly revenue
- maximum monthly revenue
- average monthly revenue
- standard deviation of the monthly revenue

for each product, ProductName

In [ ]:
-- SQL request(s)​​​​​​‌​‌​​‌‌​​​‌‌​‌​​​‌‌​‌​​‌‌ below
WITH Revenues AS (
    SELECT 
        SUM(S.Revenue) AS Revenue 
        , P.PRODUCTNAME 
        , date_trunc('month', S.OrderDate) AS OrderMonth
    FROM 
        Subscriptions AS S
            INNER JOIN 
        Products AS P
            ON 
        S.PRODUCTID = P.PRODUCTID
    WHERE 
        S.OrderDate BETWEEN '2022-01-01' AND '2022-12-31'
    GROUP By 
        P.PRODUCTNAME 
        , OrderMonth
)
SELECT 
--     *
    ProductName
    , MIN(REVENUE) AS MIN_REV
    , MAX(REVENUE) AS MAX_REV
    , AVG(REVENUE) AS AVG_REV
    , STDDEV(REVENUE) AS STD_DEV_REV
FROM 
    Revenues
GROUP BY 
    Revenues.ProductName

Task 2: Variable Distributions with CTEs

Use a CTE and the frontendeventlog table to find the distribution of users across the number of times the email link was clicked per user. 

In [ ]:
WITH email_link_clicks AS (
    SELECT 
        userid
        , COUNT(*) AS num_link_clicks
    FROM   
        frontendeventlog
    WHERE 
        eventid = 5
    GROUP BY 
        userid
)

SELECT 
    num_link_clicks
    , COUNT(*) AS num_users
FROM 
    email_link_clicks
GROUP BY 
    num_link_clicks

Task 3: Payment Funnel Analysis with Multiple CTES

Count the number of subscriptions in each paymentfunnelstage by incorporating the maxstatus reached and currentstatus per subscription. Use the paymentstatuslog and subscriptions tables. 

Funnel:
1. User opens widget
2. User types in CC info
3. User clicks submit to complete payment
4. Product sends data to 3rd party payment processing
5. Payment company completes transaction and reports back 'complete'

Status IDs
0. Error
1. PaymentWidgetOpened
2. PaymentEntered
3. PaymentSubmitted
4. PaymentSuccess
5. Complete

In [ ]:
-- Need a case switch to set up the funnel
-- Status 0 represents an error

CASE 
    WHEN maxstatus = 1 THEN 'PaymentWidgetOpened'
    WHEN maxstatus = 2 THEN 'PaymentEntered'
    WHEN maxstatus = 3 AND currentstatus = 0 THEN 'User Error with Payment Submission'
    WHEN maxstatus = 3 AND currentstatus != 0 THEN 'Payment Submitted'
    WHEN maxstatus = 4 AND currentstatus = 0 THEN 'Payment Processing Error with Vendor'
    WHEN maxstatus = 4 AND currentstatus !=0 THEN 'Payment Success'
    WHEN maxstatus = 5 THEN 'Complete'
    WHEN maxstatus IS NULL THEN 'User did not start payment process'
    END AS paymentfunnelstage

In [ ]:
WITH Max_Status_Reached AS    ( 
    SELECT SubscriptionID,
             MAX(StatusID) AS maxstatus
     FROM 
        PaymentStatusLog
     GROUP BY 
        SubscriptionID
)    
,
PaymentFunnelStages AS
    ( SELECT
            Subs.SubscriptionID, 
            case
                 when maxstatus = 1 then 'PaymentWidgetOpened'
                 when maxstatus = 2 then 'PaymentEntered'
                 when maxstatus = 3
                      and currentstatus = 0 then 'User Error with Payment Submission'
                 when maxstatus = 3
                      and currentstatus != 0 then 'Payment Submitted'
                 when maxstatus = 4
                      and currentstatus = 0 then 'Payment Processing Error with Vendor'
                 when maxstatus = 4
                      and currentstatus != 0 then 'Payment Success'
                 when maxstatus = 5 then 'Complete'
                 when maxstatus is null then 'User did not start payment process'
             end as paymentfunnelstage
     FROM 
        Subscriptions Subs
    LEFT JOIN
        Max_Status_Reached MS    
    ON Subs.SubscriptionID = MS.SubscriptionID
)


SELECT paymentfunnelstage,
       COUNT(SubscriptionID) AS subscriptions
FROM PaymentFunnelStages
GROUP BY paymentfunnelstage;



Creating Binary Columns with CASE 

Flagging upsell opportunities with the sales team. They want to reach customers meeting either of the following conditions, 'upsell_opportunity': 

1. Have at least 5000 registered users

2. Only have 1 product subscription

Create a report using the 'subscriptions' table that contains
- customerid
- totual number of products for that customer, num_products
- total number of users for that customer, total_users
- Binary column flags 1 for those who meet the upsell_opportunity conditions

In [ ]:
SELECT customerID,
       COUNT(productID) as num_products ,
       SUM(NumberofUsers) as total_users ,
       CASE
           WHEN (COUNT(productID) = 1
                 OR SUM(NumberofUsers) >= 5000) THEN 1
           ELSE 0
       END upsell_opportunity
FROM subscriptions
GROUP BY CustomerID

Pivoting Rows into aggregated columns using CASE

Tracking user activity with frontend events. We want to track user activity and ticket submissions on the customer support page since they could be positively or negatively impacted by design changes. We want to track: 

- When user views the help center page, ViewedHelpCenterPage
- When user clicks on FAQ link, ClickedFAQs
- When user clicks the contact customer support button, ClickedContactSupport
- When user clicks the submit ticket button, SubmittedTicket

Task: 
Using frontendeventlog table and CASE, count the number of times a user completes the following events: 

- ViewedHelpCenterPage (eventid = 1)
- ClickedFAQs (eventid = 2)
- ClickedContactSupport (eventid = 3)
- SubmittedTicket (eventid = 4)

Filter events with eventtype = 'Customer Support' from the frontendeventdefinitions tavbel to pull only events related to customer support

In [ ]:
SELECT UserID,
       SUM( CASE
                WHEN EventID = 1 THEN 1
                ELSE 0 -- WHEN EventID = 4 THEN SubmittedTicket

            END ) AS ViewedHelpCenterPage,
       SUM( CASE
                WHEN EventID = 2 THEN 1
                ELSE 0
            END) AS ClickedFAQs,
       SUM( CASE
                WHEN EventID = 3 THEN 1
                ELSE 0
            END ) AS ClickedContactSupport,
       SUM( CASE
                WHEN EventID = 4 THEN 1
                ELSE 0
            END ) AS SubmittedTicket
FROM 
    FrontendEventLog el
JOIN
    frontendeventdefinitions def
ON el.EventId = def.eventId
WHERE def.EventType = 'Customer Support'
GROUP BY UserID

In [ ]:
--Alternatively

SELECT 
    UserID 
    , (CASE WHEN el.EventId = 1 THEN 1 ELSE 0 END) AS ViewedHelpCenterPage
    , (CASE WHEN el.EventId = 2 THEN 1 ELSE 0 END) AS ClickedFAQs
    , (CASE WHEN el.EventId = 3 THEN 1 ELSE 0 END) AS ClickedContactSupport
    , (CASE WHEN el.EventId = 4 THEN 1 ELSE 0 END) AS SubmittedTicket
FROM 
    FrontendEventLog el
JOIN 
    frontendeventdefinitions def 
    ON el.EventID = def.EventId
WHERE 
    def.EventType = 'Customer Support'
GROUP BY 
    el.UserID


Combine product tables with UNION

Growth Officer wants to reduce churn and wants to launch marketing experiments to drive renewal. Different products are not currently in the same table, so there are currently 2 product tables. 

Task: Count the number of active subscriptions, active = 1, that will expire in each year. 

Aggregate the number of subscriptions each year as all subscriptions using a CTE

In [ ]:
-- SQL request(s)​​​​​​‌​‌​​‌‌​‌​‌‌​‌​‌​‌​‌​‌​‌‌ below
WITH all_subscriptions AS
    ( SELECT *
     FROM SubscriptionsProduct1
     WHERE Active = 1
     UNION SELECT *
     FROM SubscriptionsProduct2
     WHERE Active = 1 )
select date_trunc('year', expirationdate) as exp_year,
       count(*) as subscriptions
from all_subscriptions
group by date_trunc('year', expirationdate)

Unpivoting Columns into Rows using UNION

Analyzing Subscription Cancelation Reasons
Task: Use the cancelations table to cacluate the percent of canceled subscriptions that reported "expensive" as one of the reasons. 

In [ ]:
with all_cancelation_reasons as
    ( SELECT SubscriptionID, CancelationReason1 AS cancelationreason
     FROM Cancelations
     UNION ALL SELECT SubscriptionID, CancelationReason2 AS cancelationreason
     FROM Cancelations
     UNION ALL SELECT SubscriptionID, CancelationReason3 AS cancelationreason
     FROM Cancelations )
select cast(count( case
                       when cancelationreason = 'Expensive' then subscriptionid
                   end) as float) /count(distinct subscriptionid) as percent_expensive
from all_cancelation_reasons ;

Using Self Joins to Pull Hierarchical Relationships

Employee/Manager name data
Create an email list from the employees table that includes the following columns for all employees in the sales department: 

- employeeid
- employee_name
- manager_name
- contact_email (manager if avail, otherwise employee)

In [ ]:
SELECT e.EmployeeId ,
       e.Name AS employee_name ,
       mgr.Name AS Manager_Name,
       CASE
           WHEN mgr.email IS NOT NULL THEN mgr.email
           ELSE e.email
       END AS contact_email
FROM employees e
LEFT JOIN employees mgr ON e.ManagerID = mgr.EmployeeID
WHERE e.Department = 'Sales'

Using Self Joins to Compare Rows within the Same Table

Comparing MoM revenue (month-over-month)
Task: 

Using a CTE for monthly_revenue, pull a report that includes

- current_month
- previous_month
- current_revenue
- previous_revenue

- Only pull rows where the monthly revenue for the current month is greater than the revenue from the previous month
- Filter the data so that the date difference in months between the current and previous month is 1

In [ ]:
WITH monthly_revs AS
    ( select date_trunc('month', orderdate) as order_month,
             sum(revenue) as monthly_revenue
     from subscriptions
     group by date_trunc('month', orderdate))
SELECT current.order_month Current_Month ,
       previous.order_month Previous_Month ,
       current.monthly_revenue Current_Revenue ,
       previous.monthly_revenue Previous_Revenue
FROM monthly_revs current
JOIN monthly_revs previous
WHERE current.monthly_revenue > previous.monthly_revenue
    AND datediff('month', previous.order_month, current.order_month) = 1;